In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gdown
import zipfile
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [ ]:
file_id = '1-7mrmHfMxh9jvJ60b0b274wFcLpdkDz5'
destination = 'downloaded_file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

with zipfile.ZipFile(destination, 'r') as zip_ref:
     zip_ref.extractall('unzip_con')

print("Download and extraction complete.")

Downloading...
From (original): https://drive.google.com/uc?id=1-7mrmHfMxh9jvJ60b0b274wFcLpdkDz5
From (redirected): https://drive.google.com/uc?id=1-7mrmHfMxh9jvJ60b0b274wFcLpdkDz5&confirm=t&uuid=0039570e-b888-46d7-8a6b-aa07d79ddcab
To: /content/downloaded_file.zip
100%|██████████| 1.09G/1.09G [00:17<00:00, 63.8MB/s]


Download and extraction complete.


In [ ]:
root_path = "/content/unzip_con/food/train"
data_dict = {
    "img_path": [],
    "label": []
}

for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)

    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                data_dict["img_path"].append(file_path)
                data_dict["label"].append(folder_name)

fdf = pd.DataFrame(data_dict)
df = pd.get_dummies(fdf, columns=["label"])
df = df.replace({False : 0, True:1})


/tmp/ipython-input-3020473280.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({False : 0, True:1})


In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
)

In [ ]:
def df_to_tf_dataset(
    df,
    base_dir="",
    img_size=(256,256),
    batch_size=32,
    shuffle=True,
    augment=False,
    treat_neg_one_as_zero=True,
    cache=False
):
    paths = (base_dir + df.iloc[:, 0].astype(str)).values
    labels = df.iloc[:, 0:].values.astype(np.float32)

    if treat_neg_one_as_zero:
        labels[labels == -1] = 0.0

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), reshuffle_each_iteration=True)

    def _parse(path, label):
        image_bytes = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image_bytes, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, img_size)

        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, max_delta=0.05)
            image = tf.image.random_contrast(image, lower=0.95, upper=1.05)

        return image, label

    ds = ds.map(lambda p, l: _parse(p, l), num_parallel_calls=tf.data.AUTOTUNE)

    if cache:
        ds = ds.cache()

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = df_to_tf_dataset(df_train,
                            base_dir="",
                            img_size=(256,256),
                            batch_size=32,
                            shuffle=True,
                            augment=True,
                            treat_neg_one_as_zero=False,
                            cache=False)

val_ds = df_to_tf_dataset(df_val,
                          base_dir="",
                          img_size=(256,256),
                          batch_size=32,
                          shuffle=True,
                          augment=False,
                          treat_neg_one_as_zero=False,
                          cache=False)

In [ ]:
def SepBlock(x, filters, strides=1):
    shortcut = x
    x = layers.SeparableConv2D(filters, 3, strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.SeparableConv2D(filters, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if strides == 1 and shortcut.shape[-1] == x.shape[-1]:
        x = layers.Add()([shortcut, x])
    x = layers.ReLU()(x)
    return x

def build_lightweight_cnn(input_shape=(256,256,3), num_classes=21, dropout_rate=0.3):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)                 # normalize in [0,1]

    x = layers.Conv2D(32, 3, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = SepBlock(x, 32, strides=1)
    x = SepBlock(x, 64, strides=2)
    x = SepBlock(x, 64, strides=1)
    x = SepBlock(x, 128, strides=2)
    x = SepBlock(x, 128, strides=1)
    x = SepBlock(x, 256, strides=2)
    x = SepBlock(x, 256, strides=1)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name="light_cnn")
    return model

num_classes = 21
model = build_lightweight_cnn(num_classes=num_classes)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=["accuracy", keras.metrics.TopKCategoricalAccuracy(k=3, name="top3")]
)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=22,
    )

Epoch 1/22


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) NOT_FOUND:  0.0; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) NOT_FOUND:  0.0; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_56643]

In [ ]:
model.save_weights("my_model2.weights.h5")

In [ ]:
loaded = build_lightweight_cnn(num_classes=21)

loaded.load_weights("/content/my_model2.weights.h5")

In [ ]:
test_dict = {
    "file": [],
}
for file_name in os.listdir("/content/unzip_con/food/test"):
    file_path = os.path.join("/content/unzip_con/food/test", file_name)
    if os.path.isfile(file_path):
        test_dict["file"].append(file_name)

test_df = pd.DataFrame(test_dict)
test_df

,file
0,5AXF8CDVP5G2S.jpg
1,YRUUU233W8Z74.jpg
2,HP0IIBYL1L8SS.jpg
3,04QB5R1NMZISG.jpg
4,MU00BGORQKM5A.jpg
...,...
4271,PX9NW959JVI9B.jpg
4272,6PYF03GYRLYUC.jpg
4273,6R5FHW54DK19F.jpg
4274,M2YNJVJAXGZGF.jpg


In [ ]:
def df_to_tf_dataset_test(
    df,
    base_dir="",
    img_size=(256,256),
    batch_size=32,
):
    paths = (base_dir + df.iloc[:, 0].astype(str)).values
    ds = tf.data.Dataset.from_tensor_slices((paths))

    def _parse(path):
        image_bytes = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image_bytes, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, img_size)

        return image

    ds = ds.map(lambda p: _parse(p), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

test_ds = df_to_tf_dataset_test(df=test_df,
                            base_dir="/content/unzip_con/food/test/",
                            img_size=(256,256),
                            batch_size=32)

In [ ]:
class_names = sorted(fdf['label'].unique())
predictions_proba = loaded.predict(test_ds)
predicted_indices = predictions_proba.argmax(axis=1)
predicted_class_names = [class_names[i] for i in predicted_indices]
print("\nPredicted Indices:", predicted_indices)
print("Predicted Class Names:", predicted_class_names)

134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step

Predicted Indices: [ 8  8  3 ...  8  8 13]
Predicted Class Names: ['eggs', 'eggs', 'cheesecake', 'omelette', 'fish', 'hamburger', 'omelette', 'falafel', 'chocolate_cake', 'falafel', 'omelette', 'caesar_salad', 'eggs', 'omelette', 'fish', 'cheesecake', 'omelette', 'spaghetti', 'omelette', 'cheese_sandwich', 'eggs', 'ice_cream', 'omelette', 'eggs', 'chicken_curry', 'eggs', 'falafel', 'omelette', 'spaghetti', 'chocolate_cake', 'caesar_salad', 'sushi', 'falafel', 'chicken_curry', 'lasagna', 'falafel', 'cheesecake', 'spaghetti', 'falafel', 'french_fries', 'lasagna', 'fish', 'chocolate_cake', 'cheesecake', 'cheese_sandwich', 'baklava', 'baklava', 'omelette', 'eggs', 'spaghetti', 'hot_dog', 'sushi', 'omelette', 'french_fries', 'sushi', 'cheesecake', 'chocolate_cake', 'pizza', 'cheesecake', 'cheesecake', 'cheese_sandwich', 'cheesecake', 'fish', 'fish', 'chicken_wings', 'lasagna', 'baklava', 'spaghetti', 'hot_dog', 'falafel', 'spaghetti', 'cheese_sandw

In [ ]:
submission2 = pd.DataFrame({
    'file': test_df['file'],
    'prediction': predicted_class_names,
})

In [ ]:
submission

,file,predicted
0,5AXF8CDVP5G2S.jpg,eggs
1,YRUUU233W8Z74.jpg,eggs
2,HP0IIBYL1L8SS.jpg,cheesecake
3,04QB5R1NMZISG.jpg,omelette
4,MU00BGORQKM5A.jpg,fish
...,...,...
4271,PX9NW959JVI9B.jpg,omelette
4272,6PYF03GYRLYUC.jpg,spaghetti
4273,6R5FHW54DK19F.jpg,eggs
4274,M2YNJVJAXGZGF.jpg,eggs


In [ ]:
submission2.to_csv('submission2.csv', index=False)